In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math as m
import scipy as sp 
import seaborn as sns 
import linearmodels as lm 
import statsmodels.api as sm 
import matplotlib as mpl 
import statsmodels.formula.api as smf 

from scipy import stats
from scipy.stats import bootstrap

from sklearn.utils import resample 
from sklearn.decomposition import PCA  
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
mpl.rcParams.update(mpl.rcParamsDefault)

In [27]:
fyr = pd.read_csv('downloads\Helsfyr_data_2023-10-31_to_2024-11-01.csv', delimiter=';')

In [23]:
fyr['Fra'] = pd.to_datetime(fyr['Fra'], utc=True, errors='coerce')
fyr['Til'] = pd.to_datetime(fyr['Til'], utc=True, errors='coerce')

fyr['Fra'] = fyr['Fra'].dt.tz_convert('Europe/Oslo')
fyr['Til'] = fyr['Til'].dt.tz_convert('Europe/Oslo')

In [24]:
fyr['Dato'] = pd.to_datetime(fyr.Dato)
fyr["Fra tidspunkt"] = pd.to_datetime(fyr['Fra tidspunkt']).dt.strftime('%H:%M:%S')
fyr["Til tidspunkt"] = pd.to_datetime(fyr['Til tidspunkt']).dt.strftime('%H:%M:%S')


C:\Users\alepa\AppData\Local\Temp\ipykernel_26500\472647116.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fyr["Fra tidspunkt"] = pd.to_datetime(fyr['Fra tidspunkt']).dt.strftime('%H:%M:%S')
C:\Users\alepa\AppData\Local\Temp\ipykernel_26500\472647116.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fyr["Til tidspunkt"] = pd.to_datetime(fyr['Til tidspunkt']).dt.strftime('%H:%M:%S')


In [25]:
fyr.groupby("Felt").count()

,Trafikkregistreringspunkt,Navn,Vegreferanse,Fra,Til,Dato,Fra tidspunkt,Til tidspunkt,Trafikkmengde,Dekningsgrad (%),...,Ikke gyldig lengde,Lengdekvalitetsgrad (%),"< 5,6m",">= 5,6m","5,6m - 7,6m","7,6m - 12,5m","12,5m - 16,0m",">= 16,0m","16,0m - 24,0m",">= 24,0m"
Felt,,,,,,,,,,,,,,,,,,,,,
1,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808,...,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808
2,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808,...,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808
3,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808,...,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808
4,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808,...,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808
Totalt,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808,...,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808
Totalt i retning Alnabru,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808,...,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808
Totalt i retning Sentrum,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808,...,8808,8808,8808,8808,8808,8808,8808,8808,8808,8808


In [26]:
fyr

,Trafikkregistreringspunkt,Navn,Vegreferanse,Fra,Til,Dato,Fra tidspunkt,Til tidspunkt,Felt,Trafikkmengde,...,Ikke gyldig lengde,Lengdekvalitetsgrad (%),"< 5,6m",">= 5,6m","5,6m - 7,6m","7,6m - 12,5m","12,5m - 16,0m",">= 16,0m","16,0m - 24,0m",">= 24,0m"
0,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31 00:00:00+01:00,2023-10-31 01:00:00+01:00,2023-10-31,00:00:00,01:00:00,1,0,...,0,"100,00",0,0,0,0,0,0,0,0
1,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31 00:00:00+01:00,2023-10-31 01:00:00+01:00,2023-10-31,00:00:00,01:00:00,2,39,...,0,"100,00",35,4,3,0,0,1,1,0
2,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31 00:00:00+01:00,2023-10-31 01:00:00+01:00,2023-10-31,00:00:00,01:00:00,3,142,...,1,"99,30",105,36,16,9,3,8,6,2
3,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31 00:00:00+01:00,2023-10-31 01:00:00+01:00,2023-10-31,00:00:00,01:00:00,4,179,...,0,"100,00",123,56,6,10,4,36,33,3
4,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31 00:00:00+01:00,2023-10-31 01:00:00+01:00,2023-10-31,00:00:00,01:00:00,Totalt i retning Alnabru,218,...,0,"100,00",158,60,9,10,4,37,34,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61651,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2024-10-31 23:00:00+01:00,2024-11-01 00:00:00+01:00,2024-10-31,23:00:00,00:00:00,3,268,...,0,"100,00",230,38,9,11,5,13,13,0
61652,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2024-10-31 23:00:00+01:00,2024-11-01 00:00:00+01:00,2024-10-31,23:00:00,00:00:00,4,207,...,1,"99,52",161,45,8,7,4,26,21,5
61653,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2024-10-31 23:00:00+01:00,2024-11-01 00:00:00+01:00,2024-10-31,23:00:00,00:00:00,Totalt i retning Alnabru,363,...,1,"99,72",306,56,11,12,4,29,24,5
61654,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2024-10-31 23:00:00+01:00,2024-11-01 00:00:00+01:00,2024-10-31,23:00:00,00:00:00,Totalt i retning Sentrum,333,...,0,"100,00",291,42,13,11,5,13,13,0


In [ ]:
keep = [
    "Trafikkregistreringspunkt", "Navn", "Vegreferanse", "Dato", 
    "Fra tidspunkt", "Til tidspunkt", "Felt", "Trafikkmengde",
    "< 5,6m", ">= 5,6m", "5,6m - 7,6m", "7,6m - 12,5m", 
    "12,5m - 16,0m", ">= 16,0m", "16,0m - 24,0m", ">= 24,0m"
]
# Filter rows based on 'Felt' values
felt_values = ["Totalt", "Totalt i retning Alnabru", "Totalt i retning Sentrum"]
fyr = fyr[fyr["Felt"].isin(felt_values)]
fyr = fyr[keep]
# Replace '-' with 0 and convert relevant columns to numeric
length_columns = [
    "< 5,6m", "5,6m - 7,6m", "7,6m - 12,5m", "12,5m - 16,0m",
    ">= 16,0m", "16,0m - 24,0m", ">= 24,0m"
]

fyr[length_columns] = fyr[length_columns].replace('-', 0).apply(pd.to_numeric)

# Calculate 'light' and 'heavy' vehicle counts
fyr['Light Vehicles'] = fyr["< 5,6m", ">= 5,6m"]
fyr['Heavy Vehicles'] = fyr[length_columns[1:]].sum(axis=1)

# Drop the original length columns
df = df.drop(columns=length_columns)

# Save the processed data to a new CSV file
df.to_csv('processed_traffic_data.csv', index=False, sep=';')

print("Processing complete. The data has been saved to 'processed_traffic_data.csv'.")


KeyError: "['Fra tidspunkt', 'Til tidspunkt'] not in index"

In [13]:
fyr

,Trafikkregistreringspunkt,Navn,Vegreferanse,Dato,Fra tidspunkt,Til tidspunkt,Felt,Trafikkmengde,"< 5,6m",">= 5,6m","5,6m - 7,6m","7,6m - 12,5m","12,5m - 16,0m",">= 16,0m","16,0m - 24,0m",">= 24,0m"
4,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31,00:00:00,01:00:00,Totalt i retning Alnabru,218,158,60,9,10,4,37,34,3
5,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31,00:00:00,01:00:00,Totalt i retning Sentrum,142,105,36,16,9,3,8,6,2
6,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31,00:00:00,01:00:00,Totalt,360,263,96,25,19,7,45,40,5
11,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31,01:00:00,02:00:00,Totalt i retning Alnabru,161,100,61,8,5,2,46,40,6
12,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2023-10-31,01:00:00,02:00:00,Totalt i retning Sentrum,162,126,36,5,13,3,15,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61647,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2024-10-31,22:00:00,23:00:00,Totalt i retning Sentrum,521,453,67,13,20,4,30,26,4
61648,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2024-10-31,22:00:00,23:00:00,Totalt,1066,943,122,22,35,10,55,49,6
61653,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2024-10-31,23:00:00,00:00:00,Totalt i retning Alnabru,363,306,56,11,12,4,29,24,5
61654,33808V625649,"Helsfyr 1,2,3,4",EV6 S16D20 m2636,2024-10-31,23:00:00,00:00:00,Totalt i retning Sentrum,333,291,42,13,11,5,13,13,0


In [14]:
fyr['Time Range'] = fyr['Fra tidspunkt'].astype(str) + " - " + fyr['Til tidspunkt'].astype(str)

In [15]:
fyr = fyr.drop(['Fra tidspunkt', 'Til tidspunkt'], axis=1)

In [16]:
fyr.groupby(['Dato', 'Time Range','Felt']).sum()

Trafikkregistreringspunkt  \
Dato       Time Range          Felt                                                 
2023-10-31 00:00:00 - 01:00:00 Totalt                                33808V625649   
                               Totalt i retning Alnabru              33808V625649   
                               Totalt i retning Sentrum              33808V625649   
           01:00:00 - 02:00:00 Totalt                                33808V625649   
                               Totalt i retning Alnabru              33808V625649   
...                                                                           ...   
2024-10-31 22:00:00 - 23:00:00 Totalt i retning Alnabru              33808V625649   
                               Totalt i retning Sentrum              33808V625649   
           23:00:00 - 00:00:00 Totalt                                33808V625649   
                               Totalt i retning Alnabru              33808V625649   
                               Totalt i retning Sentrum              33808V625649   

                                                                    Navn  \
Dato       Time Range          Felt                                        
2023-10-31 00:00:00 - 01:00:00 Totalt                    Helsfyr 1,2,3,4   
                               Totalt i retning Alnabru  Helsfyr 1,2,3,4   
                               Totalt i retning Sentrum  Helsfyr 1,2,3,4   
           01:00:00 - 02:00:00 Totalt                    Helsfyr 1,2,3,4   
                               Totalt i retning Alnabru  Helsfyr 1,2,3,4   
...                                                                  ...   
2024-10-31 22:00:00 - 23:00:00 Totalt i retning Alnabru  Helsfyr 1,2,3,4   
                               Totalt i retning Sentrum  Helsfyr 1,2,3,4   
           23:00:00 - 00:00:00 Totalt                    Helsfyr 1,2,3,4   
                               Totalt i retning Alnabru  Helsfyr 1,2,3,4   
                               Totalt i retning Sentrum  Helsfyr 1,2,3,4   

                                                             Vegreferanse  \
Dato       Time Range          Felt                                         
2023-10-31 00:00:00 - 01:00:00 Totalt                    EV6 S16D20 m2636   
                               Totalt i retning Alnabru  EV6 S16D20 m2636   
                               Totalt i retning Sentrum  EV6 S16D20 m2636   
           01:00:00 - 02:00:00 Totalt                    EV6 S16D20 m2636   
                               Totalt i retning Alnabru  EV6 S16D20 m2636   
...                                                                   ...   
2024-10-31 22:00:00 - 23:00:00 Totalt i retning Alnabru  EV6 S16D20 m2636   
                               Totalt i retning Sentrum  EV6 S16D20 m2636   
           23:00:00 - 00:00:00 Totalt                    EV6 S16D20 m2636   
                               Totalt i retning Alnabru  EV6 S16D20 m2636   
                               Totalt i retning Sentrum  EV6 S16D20 m2636   

                                                         Trafikkmengde < 5,6m  \
Dato       Time Range          Felt                                             
2023-10-31 00:00:00 - 01:00:00 Totalt                              360    263   
                               Totalt i retning Alnabru            218    158   
                               Totalt i retning Sentrum            142    105   
           01:00:00 - 02:00:00 Totalt                              323    226   
                               Totalt i retning Alnabru            161    100   
...                                                                ...    ...   
2024-10-31 22:00:00 - 23:00:00 Totalt i retning Alnabru            545    490   
                               Totalt i retning Sentrum            521    453   
           23:00:00 - 00:00:00 Totalt                              696    597   
                               Totalt i retning Alnabru            3

In [28]:
fyr.to_csv('Helsfyr_cleaned.csv', index=False, encoding='utf-8')